# Hard Margin SVM

In [1]:
# IMPORTS
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interact, fixed

In [71]:
# TEST DATA #1 (DATAFRAME D1)

# Create some data with 100 samples, 2 features, 1 class label, that's linearly separable:
n = 50
np.random.seed(0)
X = np.random.randn(n, 2)
y = np.random.choice([-1, 1], n)

# Shift the classes to make them linearly separable with a clear gap
X[y == 1] += 2
X[y == -1] -= 2

# Create a DataFrame with features and class labels:
D1 = pd.DataFrame(X, columns=['x1', 'x2'])
D1['class'] = y

### Idea: 

**WHAT**

SVM finds the hyperplane that <u>best</u> separates the data into two classes.
- <u>best</u> = maximizes the margin between the two classes.

**WHY**

Having a large margin means that the model is less likely to overfit the data.

<img src="img/img1.jpeg" style="width: 50%"/>



### Min-Distance between a point and Hyperplane 

Given: 

- $w^T x = -b$
- any point on the hyperplane: $x_h$
- Point of interest: $x_i$ 

Let: 

- $v = x_i - x_h$ 

Min distance: 

$$
    \begin{aligned}
        d 
        &= \frac{w}{||w||} \cdot v
        \\
        &= \frac{w^T x_i + b}{||w||}

    \end{aligned}
$$


<img src="img/img5.jpeg" style="width:50%">

In [ ]:
# DEMO: MIN DISTANCE BETWEEN POINTS AND HYPERPLANE

def signedDistPointToLine(w, b, x):
    return (np.dot(x, w) + b) / np.linalg.norm(w)

# now find the point with the smallest distance to the decision boundary:
def find_closest_point(w, b, X):
    # calculate the distance from each point to the decision boundary
    signed_dists = [ signedDistPointToLine(w, b, x) for x in X ]
    # find the point with the smallest distance
    i = np.argmin(np.abs(signed_dists))
    return X[i], signed_dists[i]

# PLOTTING:

# Now that the data is created, create a decision boundary and plot it:
def plot_decision_boundary(w, b, D: pd.DataFrame):
    x = np.linspace(-3, 3, 100)
    y = (-w[0] * x - b) / w[1]
    plt.plot(x, y, 'k--')
    plt.xlim(-3, 3)
    plt.ylim(-3, 3)
    # plot the data: use first 2 columns of D as x and y, and color by class:
    plt.scatter(D['x1'], D['x2'], c=D['class'], cmap='coolwarm')


def closestPointFromDecisionBoundary(w1=1, w2=1, b: int=0):
    w = [w1, w2]
    cp, signed_dist = find_closest_point(w, b, D1.iloc[:, :2].values)

    v = ( w / np.linalg.norm(w) ) * -signed_dist    # vector between closest point and decision boundary
    pol = cp + v                            # point on the decision boundary

    # PLOT: data
    plt.scatter(D1['x1'], D1['x2'], c=D1['class'], cmap='coolwarm')
    # PLOT: decision boundary
    plot_decision_boundary(w, b, D1)
    # PLOT: closest point to the line
    plt.scatter(cp[0], cp[1], c='black', s=100)
    # PLOT: line btwn cp & pol:
    plt.plot([pol[0], cp[0]], [pol[1], cp[1]], 'k--')


interact (closestPointFromDecisionBoundary, w1=(-3, 3, 0.1), w2=(-3, 3, 0.1), b=(-2,2,0.1))


### The Margin 

"Margin" = distance from hyperplane (decision boundary) to the closest point. 

$$
    \begin{aligned}
        \gamma 
        &= \text{min}_i( y_i \frac{w^T x_i + b}{||w||})
        \\
        &= \frac{1}{||w||} \text{min}_i( y_i (w^T x_i + b))
    \end{aligned}
$$

- $y_i \in [-1, 1]$ (the class) converts the distance to be unsigned.

**Diagram:**

<img src="img/img2.jpeg" style="width: 30%"/>

### "Max Margin" Classifier Problem 

We need to maximize the margin *(the min distance)*.

$$
    \begin{aligned}
        \hat{w}, \hat{b} 
        &= \text{argmax}_{w,b} \gamma
        \\
        &= \text{argmax}_{w,b} ( 
            \frac{1}{||w||} \text{min}_i( y_i (w^T x_i + b))
        )
    \end{aligned}
$$

<img src="img/img3.jpeg" style="width: 50%">

In [ ]:
# DEMO: MARGIN

def marginFromDecisionBoundary(w1=1, w2=1, b: int=0):
    w = [w1, w2]
    cp, signed_dist = find_closest_point(w, b, D1.iloc[:, :2].values)
    margin = np.abs(signed_dist)

    # PLOT: data
    plt.scatter(D1['x1'], D1['x2'], c=D1['class'], cmap='coolwarm')

    # PLOT: decision boundary
    plot_decision_boundary(w, b, D1)

    # PLOT: closest point to the line
    plt.scatter(cp[0], cp[1], c='black', s=100)

    # PLOT: margin
    x_fill = np.arange(-3, 3, 0.01)
    y_fill =  - (w[0]*x_fill + b) / w[1]    # y-values on decision boundary
    plt.fill_between(x_fill, y_fill - margin, y_fill + margin, color='grey', alpha=0.5)


interact (marginFromDecisionBoundary, w1=(-3, 3, 0.1), w2=(-3, 3, 0.1), b=(-2,2,0.1))


### Scaling Parameters does not change the Decision Boundary

$w \rarr \alpha w$, $b \rarr \alpha b$

For each $x_i$, the decision boundary intersects the $x_i$-axis at $x_i = -\frac{b}{w_i} = -\frac{\alpha \cdot b}{\alpha \cdot w_i}$. 


The distance between any point and the boundary does not change either.

$$
    \begin{aligned}
        d 
        &= 
        \frac{w^T x_i + b}{||w||}
        \\
        &=
        \frac{\red{\alpha} w^T x_i + \red{\alpha}b}{||\red{\alpha}w||}
        \\
        &=
        \frac{\red{\alpha}}{\red{\alpha}}
        \frac
            { w^T x_i + b}
            {||w||}
    \end{aligned}
$$

In [ ]:
# DEMO: Scaling Parameters

def scaleParameters(scale=1):
    w=[1,1]
    b=1
    marginFromDecisionBoundary(w1=w[0]*scale, w2=w[1]*scale, b=-b*scale)
    plt.legend([
        f'x1: -{scale}*b / {scale}*w1 = {-b/w[0]}',
        f'x2: -{scale}*b / {scale}*w2 = {-b/w[1]}',
    ])

interact (scaleParameters, scale=(1, 10, 1))

### Restating the Objective (Constrained Optimization)

Create 2 equations: 

1. Positive class: $w^T x_i + b = 1$
2. Negative class: $w^T x_i + b = -1$

i.e. Rescale the equation to have the margin = +1/-1

<img src="img/img4.jpeg" style="width: 40%"/>


We turn our nested (max over min) optimization problem into a single (constrained) optimization problem (easier to compute):

$$
    \begin{aligned}
        \hat{w}, \hat{b} 
        &= \text{argmax}_{w,b} \gamma
        \\
        &= \text{argmax}_{w,b} 
            \frac{1}{||w||} \text{min}_i y_i (w^T x_i + b)
        \\
        &= \text{argmax}_{w,b} 
            \frac{1}{||w||} 
            , \ \ \ \ \  \text{subject to: }  y_i (w^T x_i + b) \ge 1
        \\
        &= \text{argmin}_{w,b} 
            \frac{1}{2} ||w||^2
             , \ \ \ \text{subject to: }  y_i (w^T x_i + b) \ge 1
    \end{aligned}
$$

**Note:** We take half the squared weight to make the math easier later on. We can do this since the margin is always positive. 

# Solving Constraint Optimization Problem
### <span style="color: grey;">Using Lagrange Multipliers (the Dual Problem)</span>


## Lagrangian Dual Problem 

The dual problem helps solve constrained problems by transforming them into a new form.

The Lagrangian $L(x,\lambda)$ is a function that incorporates both the objective function and the constraint:

$$
    L(x, \lambda) = f(x) - \lambda (x - c)
$$

where $\lambda\ge 0$ is the Lagrange multiplier that enforces the constraint.


The dual problem involves:

$$
        \text{min}_x
        \text{max}_{\lambda}
        L(x, \lambda) 
$$

1. Minimizing  $L(x,\lambda)$ with respect to $x$
2. Maximizing it with respect to $\lambda$


### Example 1

Minimize: $f(x)=0.2x^2-x+1$, subject to $x \ge \red{5}$

Let: 

- $L(x, \lambda) = f(x) - \lambda (x-\red{5})$
- $\lambda \ge 0$

---

1. Minimize over $x$:

$$
    \frac{\partial L}{\partial x} = 0.4x - 1 - \lambda
$$

Let: 

$$
    0.4x - 1 - \lambda = 0
$$

2. Maximize over $\lambda$:

$$
    \frac{\partial L}{\partial \lambda} = x - 5
$$

Let: 

$$
    x - 5 = 0
    \\
    \implies x = 5
    
$$


3. Combine: 

$$
    \lambda = 0.4 (5) - 1 = 1
$$


As $\lambda \ge 0$, $x=5$ is a valid solution.

--- 



In [ ]:
# DEMO: Solving the Dual problem

x = np.arange(-5, 10, 0.1)
y = 0.2*(x**2) - x + 1

def findMin(x: np.ndarray, y: np.ndarray, x_min: int):
    i = np.argmax(x >= x_min)
    i_min = np.argmin(y[i:])
    return x[i + i_min], y[i + i_min]

def getLagrangian(x: np.ndarray, y:np.ndarray, minX: int, LM: int):
    return y - (LM*(x-minX))


def plotDualProblem(y: np.ndarray, minX: int, LM=1):
    L = y - (LM*(x-minX))
    plt.plot(x, y)
    plt.plot(x,L)
    plt.vlines(minX, -5, 10, colors='black', linestyles='dashed')
    # ensure that the x-axis is fixed:
    plt.xlim(-5, 10)
    plt.ylim(-5, 10)
    # get the min point on y:
    min_x, min_y = findMin(x=x, y=y, x_min=minX)
    plt.scatter(min_x, min_y, c='black', s=100)


interact (plotDualProblem, y=fixed(y), minX=(-5, 10, 0.1), LM=(0, 10, 0.1))

### Example 2

$f(x)=0.2x^2-x+1$, subject to $x \ge \red{-3}$

=> 

- $L(x, \lambda) = f(x) - \lambda (x+\red{3})$
- $\lambda \ge 0$

---

1. Minimize over $x$:

$$
    \frac{\partial L}{\partial x} = 0.4x - 1 - \lambda
$$

Let: 

$$
    0.4x - 1 - \lambda = 0
$$

2. Maximize over $\lambda$:

$$
    \frac{\partial L}{\partial \lambda} = x - 5
$$

Let: 

$$
    x +3 = 0
    \\
    \implies x = -3
    
$$


3. Combine: 

$$
    \lambda = 0.4 (-3) - 1 = -2.2
$$


As $\lambda \le 0$, $x=-3$ is NOT a valid solution.


4. Let $\lambda = 0$, solve for $x$:

$$
    x=\frac{1}{0.4}=2.5
$$

--- 



### Explanation

The solution to the dual problem provides insight into both the optimal value of the variable `x` in the original problem and how \"binding\" the constraint is, represented by the Lagrange multiplier `λ`. Here’s how to intuitively connect the dual solution to the original problem

#### 1. **Understanding `x = 5`** (from the dual problem):

- The constraint in the original problem is `x ≥ 5`.
- The fact that the dual solution gives `x = 5` means that the constraint is **active** or **binding**. In other words, the solution occurs right at the boundary of the constraint, so `x` cannot be less than 5.
- Had the solution been `x > 5`, the constraint wouldn’t be binding, meaning the constraint wouldn’t have played a limiting role in the optimization.

#### 2. **Understanding `λ = 1`**:

- The Lagrange multiplier `λ = 1` gives the **sensitivity** of the objective function to changes in the constraint. Specifically, `λ` measures how much the optimal value of the objective function (here, `f(x)`) would increase if the constraint `x ≥ 5` were relaxed by a small amount.
- In this case, `λ = 1` means that if you relaxed the constraint by allowing `x` to be slightly smaller than 5 (e.g., `x ≥ 4.99`), the value of the objective function would improve (decrease) by 1 unit for each unit of relaxation.

#### 3. **Interpretation in the Original Problem**:

- **Solution for `x`**: From the dual problem, `x = 5` is the value of `x` that minimizes `f(x)` subject to the constraint. This directly answers the original problem: the optimal value of `x` is 5.
- **Meaning of `λ`**: `λ = 1` tells us that the constraint `x ≥ 5` is important because relaxing it would improve the objective function. If `λ = 0`, it would mean the constraint wasn’t binding, and the optimization could have proceeded without regard to the constraint.

#### Summary:

- The dual solution `x = 5` confirms that the optimal solution to the original problem occurs at the boundary of the constraint.
- `λ = 1` indicates how strongly the constraint `x ≥ 5` influences the solution—relaxing the constraint would decrease the function value (making the optimization easier).

In essence, the dual problem not only provides the solution `x = 5` but also quantifies the \"cost\" or impact of the constraint via `λ = 1`. You use this information to conclude that the best solution for the original problem is at `x = 5`, and the constraint is crucial to this outcome.

## Max-Margin Problem Using Lagrangian Dual

$$
\hat{w}, \hat{b} 
= 
\text{argmin}_{w,b} 
\frac{1}{2} ||w||^2
, \ \ \ 
\text{subject to: }  y_i (w^T x_i + b) \ge 1
$$

Notice, the constraint is for every point in the dataset. 

becomes... 

$$
\text{min}_{w,b}
\text{max}_{\blue{\lambda} \ge 0}
L(w,b,\blue{\lambda})
= 
\frac{1}{2} ||w||^2 - \sum_i \blue{\lambda}_i (y_i (w^T x_i + b) - 1)
$$


Intuition: 

- The points that are closest to the decision boundary will have the highest $\lambda_i$ values (& vice-versa).
- Their lambda values will active; used to update the weights and bias.

## Optimizing the Lagrangian Dual

$$
\begin{aligned}
    \text{min}_{w,b}
    \text{max}_{\blue{\lambda} \ge 0}
    L(w,b,\blue{\lambda})
    
    &= 
    \frac{1}{2} ||w||^2 - \sum_i \blue{\lambda}_i (y_i (w^T x_i + b) - 1)

    \\ &= 
    \frac{1}{2} ||w||^2 - \sum_i \blue{\lambda}_i y_i w^T x_i - \sum_i \blue{\lambda}_i y_i b + \sum_i \blue{\lambda}_i

\end{aligned}
$$


1. Minimize over $w$ and $b$:

$$
    \begin{aligned}
        \frac{\partial L}{\partial w} 
        &= w - \sum_i \blue{\lambda}_i y_i x_i = 0
        \\
        &\implies w = \sum_i \blue{\lambda}_i y_i x_i
    \end{aligned}
$$

- Note: any point that is not on the margin will have $\lambda_i = 0$, and those points will not contribute to the weight update.
    - => not all data points are needed to train the model.

2. Minimize over $b$:

$$
    \begin{aligned}
        \frac{\partial L}{\partial b} 
        &= - \sum_i \blue{\lambda}_i y_i = 0
        \\
        &\implies \sum_i \blue{\lambda}_i y_i = 0
    \end{aligned}
$$


3. Solve for $\lambda_i$:

<img src="img/img6.jpeg" style="width:50%"/>


4. Solve for $b$: 

$$
    y_i(w^T x_i+b) = 1, \ \ \ x_i \in S_v
    \\
    \begin{aligned}
        \implies  b &= \frac{1}{y_i} - w^T x_i
        \\ &= 
        y_i - w^T x_i
        \\ &\approx
        \frac{1}{N_{sv}} \sum_{x_i \in S_v} (y_i - w^T x_i)
    \end{aligned}

$$





In [ ]:
import cvxpy as cp

# Problem data:
m = 30   # number of constraints
n = 20   # number of variables
np.random.seed(1)
A = np.random.randn(m, n)   # random matrix
b = np.random.randn(m)      # random vector

# Construct the problem:
x = cp.Variable(n)          # lambda_i
objective = cp.Minimize(cp.sum_squares(A @ x - b)) # objective
constraints = [0 <= x, x <= 1]      # lambda_i >= 0
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(x.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print(constraints[0].dual_value)